# Analyzing outliers from data & Returning Dates

In [1]:
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import pandas_datareader.data as web
import seaborn as sns
from scipy.stats import norm
from scipy import stats
import numpy as np
import os
import time
from itertools import chain
from IPython.display import display_html


##### ETF Data Clean up ###########

class ETFDataCleanup(object):
    
    def __init__(self):
        pass

    def showNaColumns(self,df):
        s=df.isnull().sum()
        print(s[s>0])
    
    def dropNAColumns(self,df):
        return df.dropna(axis='columns')
    
    def computeDailyReturns(self,df):
        return df.pct_change().dropna()        
    
    
####### Get Data for Constituents of ETF

class ConstituentsData(ETFDataCleanup):
    
    def __init__(self,fileName=None,startdate=None,enddate=None):
        self.fileName=fileName
        self.startdate=startdate
        self.enddate=enddate
        self.constituentdata=[]
        self.constituentcloseDF=[]
        self.tickerdf=[]
        self.changeDF=[]
        self.waDF=[]
    
    def getconstituentdata(self):
        self.tickerdf = pd.read_excel(self.fileName)
        tickers=self.tickerdf['Ticker']
        self.tickerdf.set_index('Ticker',inplace=True)
        self.constituentdata =  web.DataReader(tickers,'yahoo',self.startdate,self.enddate)
        self.constituentcloseDF = self.constituentdata['Close'].iloc[:, :]
        
    def stringWeightsToFloat(self):
        self.tickerdf['Weights']=self.tickerdf['Weights'].apply(lambda x: x.replace('%','')).astype(float)
    
    def findNetAssetValue(self):
        self.waDF=self.changeDF.copy()
        for col in self.changeDF.columns:
            # Divide by 100 for weights percentage eg 23.28%
            self.waDF[col]=self.changeDF[col]*self.tickerdf['Weights'].loc[col]/100
        self.waDF['NAV']=self.waDF.sum(axis=1)

####### Get prices of ETF        
class ETFStockPrices(ETFDataCleanup):
    
    def __init__(self,etfticker=None,startdate=None,enddate=None):
        self.etfticker=etfticker
        self.startdate=startdate
        self.enddate=enddate
        self.etfdata=[]
        self.etfchangeDF=[]

        
    def getETFTickerData(self):
        self.etfdata =  web.DataReader(self.etfticker,'yahoo',self.startdate,self.enddate)
 


class ETFArbitrage(object):

    def __init__(self,etfob,waDF):
        self.navDF=pd.merge(etfob,waDF['NAV'],left_index=True,right_index=True)
        self.navDF['Date']=self.navDF.index
        self.navDF['Close']=self.navDF['Close']*100
        self.navDF['NAV']=self.navDF['NAV']*100
        del self.navDF['Date']
        self.navDF['Mispricing']=(self.navDF['Close']-self.navDF['NAV'])
        self.navDF['Z-Score'] = np.abs(stats.zscore(self.navDF['Mispricing']))
    
    def scatterplot(self):
        plt.scatter(self.navDF['Close'],self.navDF['NAV'])
        
    def scatterplot2(self):
        #change figure size for the plot
        fig_size = plt.rcParams["figure.figsize"]
        fig_size[0]=12
        fig_size[1]=6
        plt.rcParams["figure.figsize"]=fig_size

        plt.scatter(self.navDF.index,self.navDF['Mispricing'])
    
    def seaborndist(self):
        sns.distplot(self.navDF['Mispricing'].values, hist=True, kde=True, 
             bins='auto', color = 'black', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 2})
        


# Load any ETF you want

In [ ]:
tickeretf="XLK"

filename='ETFDailyData'+'/'+dt.datetime.now().strftime("%Y%m%d")+'/'+tickeretf+'.xls'
startdate=dt.datetime(2019,1,1)
enddate=dt.datetime.today()

ob=ConstituentsData(fileName=filename,startdate=startdate,enddate=enddate)
ob.getconstituentdata()
print(tickeretf)
print("Tickers with NA Values")
ob.showNaColumns(ob.constituentcloseDF)
ob.constituentcloseDF=ob.dropNAColumns(ob.constituentcloseDF)
print("***************")
print("Check for NA Values again")
ob.showNaColumns(ob.constituentcloseDF)
ob.changeDF=ob.computeDailyReturns(ob.constituentcloseDF)
print(ob.changeDF.tail(10))


ob.stringWeightsToFloat()
ob.findNetAssetValue()

print("***************")
print("Data for Constituents")
print(ob.waDF.head(5))

etfob=ETFStockPrices(etfticker=tickeretf,startdate=startdate,enddate=enddate)
etfob.getETFTickerData()
print("Show Any Empty Values")
etfob.showNaColumns(etfob.etfdata)
etfob.etfchangeDF=etfob.computeDailyReturns(etfob.etfdata['Close'])


print("***************")
print("Data for ETF stock prices")
print(etfob.etfchangeDF.head(5))

print("****************")
print("Charts")
arbob=ETFArbitrage(etfob.etfchangeDF,ob.waDF)
arbob.scatterplot()
plt.show()
arbob.scatterplot2()
plt.show()
arbob.seaborndist()
plt.show()



# List down all objects of 3 different classes

In [ ]:
print("Constituents Data Objects")
print(dir(ob))

print("*************************")
print("ETF Price Object")
print(dir(etfob))

print("*************************")
print("Arbitrage Data Object")
print(dir(arbob))

## arbob.navDF is the ETF arbitrage dataframe which tells about 
1) Daily Return of ETF<br>
2) NET Asset Value of ETF<br>
3) Mispricing <br>
4) Stocks which moved XLK ETF the most<br>

In [ ]:
print(arbob.navDF.head(5))

# Show me days with Arbitrage Opportunities of greater than 3 Standard Deviation

In [ ]:
stdthresold=2.5

daysofarbitrage=arbob.navDF[arbob.navDF['Z-Score']>stdthresold]
daysofarbitrage

## Stocks Returns, Volume & Volume Weighted Returns Standard Deviation

In [ ]:
# Good Function
class ZscoreAnlysByAttr():

    def __init__(self,data,zthresh,colname):
        self.data=data
        self.data.name=self.data.name+' '+colname
        self.zthresh=zthresh
        
    def getMispricedData(self):
        self.df=self.data.to_frame()
        self.df['Z-Score']=np.abs(stats.zscore(self.df.values.tolist()))
        self.requiredDF=self.df[self.df['Z-Score']>self.zthresh]
        return self.requiredDF

def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [ ]:
ticker ='AAPL'
weightedMovement=ob.changeDF[ticker]*ob.constituentdata['Volume'][ticker]
weightedMovement=weightedMovement.dropna()

stockVolumeStd=ZscoreAnlysByAttr(ob.constituentdata['Volume'][ticker],stdthresold,'Volume').getMispricedData()
stockReturnStd=ZscoreAnlysByAttr(ob.changeDF[ticker]*100,stdthresold,'Return').getMispricedData()
stockweightedmovement=ZscoreAnlysByAttr(weightedMovement,stdthresold,'Volume Weighted Return').getMispricedData()

In [ ]:
display_side_by_side(stockVolumeStd,stockReturnStd,stockweightedmovement)

In [ ]:
daysofarbitrage

In [ ]:
d=[list(daysofarbitrage.index),list(stockVolumeStd.index),list(stockReturnStd.index),list(stockweightedmovement.index)]
#d=[list(stockReturnStd.index),list(daysofarbitrage.index)]
set.intersection(*map(set,d))

### Find Intersection of Dates where values were > 3 std <br>
 1) daysofarbitrage - When ETF Gave 3 STD away Returns<br>
 2) stockVolumeStd - When Volume was 3 std away from normal day for Stock<br>
 3) stockReturnStd - When Daily Returns were 3 srd away from normal days<br>
 4) stockweightedmovement - When Daily Returns * Volume were 3 Stanadrd Deviation Away<br>

### Finding Dates for Mispricing Due to any of all Tickers

In [ ]:
tickers=ob.constituentcloseDF.columns
DF=pd.DataFrame()
kvpairs={}
#MispriceDF.set_index('Ticker',inplace=True)
for ticker in tickers:
    weightedMovement=ob.changeDF[ticker]*ob.constituentdata['Volume'][ticker]
    weightedMovement=weightedMovement.dropna()
    stockVolumeStd=ZscoreAnlysByAttr(ob.constituentdata['Volume'][ticker],stdthresold,'Volume').getMispricedData()
    stockReturnStd=ZscoreAnlysByAttr(ob.changeDF[ticker]*100,stdthresold,'Return').getMispricedData()
    stockweightedmovement=ZscoreAnlysByAttr(weightedMovement,stdthresold,'Volume Weighted Return').getMispricedData()
    #d=[list(daysofarbitrage.index),list(stockVolumeStd.index),list(stockReturnStd.index),list(stockweightedmovement.index)]
    d=[list(daysofarbitrage.index),list(stockweightedmovement.index)]
    lst=(list(set.intersection(*map(set,d))))
    if len(lst)>0:
        kvpairs[ticker]=lst
#kvpairs
MispriceDF=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in kvpairs.items() ]))
print(MispriceDF)

### Dropping Tickers with all NA values

In [ ]:
MispriceDF=MispriceDF.T
MispriceDF

# Adding Stocks which caused miss pricing to dataframe 

In [ ]:
# Invert a dict
class InvertDict(object):

    def invertdict(self,d):
        newdict = {}
        for key, value in d.items():
            for string in value:
                newdict.setdefault(string, []).append(key)
    
        return newdict

In [ ]:
stockscausingmispricing=InvertDict().invertdict(kvpairs)

In [ ]:
daysofarbitrage['Stocks Caused Mispricing'] = daysofarbitrage.index.to_series().map(stockscausingmispricing)

In [ ]:
daysofarbitrage